In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
selected_features = ['dst_host_same_src_port_rate',
 'is_guest_login',
 'num_failed_logins',
 'dst_host_diff_srv_rate',
 'rerror_rate',
 'logged_in',
 'serror_rate',
 'same_srv_rate',
 'protocol_type_0',
'protocol_type_1', 
 'dst_host_srv_diff_host_rate']

In [3]:
df = pd.read_csv("../../../transformed_datasets/transformed_KDD_CUPP99_2.csv")

In [4]:
feature_columns = []
for feature in df.columns:
    if df[feature].dtype == 'int64' or df[feature].dtype == 'float64':
        feature_columns.append(feature)

In [5]:
features_to_be_normalised = []
for feature in feature_columns:
    if not df[feature].min() >= 0.0 and df[feature].max() <= 1.0:
        features_to_be_normalised.append(feature)
df['label'].nunique()

23

In [6]:
# scaler = StandardScaler()
# df[features_to_be_normalised] = scaler.fit_transform(df[features_to_be_normalised])
# df.describe()

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [7]:
df = df.astype({col: 'float32' for col in df.select_dtypes(include=['int', 'float64']).columns})

In [8]:
for f in selected_features:
    print(df[f].dtype)

float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32


In [9]:
features_tensor = torch.tensor(df[selected_features].values)

In [10]:
target_tensor = torch.tensor(df['label'].values, dtype=torch.long)

In [23]:
learning_rate = 1e-3
batch_size = 64
epochs = 30

In [12]:
dataset = TensorDataset(features_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)

In [13]:
device = torch.device("mps")

In [14]:
class MultiLayerPerceptron(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 300),
            nn.ReLU(),
            nn.Linear(300, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 38),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
        

In [15]:
model = MultiLayerPerceptron().to(device)

In [16]:
loss_fn = nn.CrossEntropyLoss()

In [17]:
def train_model(dataloader, loss_fn, model, optimizer):
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Move data to the device
        prediction = model(X)
        loss = loss_fn(prediction, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    

In [18]:
def test_model(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # Move data to the device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
t0 = time.time()

In [28]:
for i in range(epochs):
    print(i)
    train_model(dataloader, loss_fn, model, optimizer)
    test_model(dataloader, model, loss_fn)

0
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.038312 

1
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.037904 

2
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.037512 

3
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.037150 

4
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.036788 

5
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.036469 

6
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.036128 

7
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.035831 

8
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.035537 

9
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.035268 

10
Test Error: 
 Accuracy: 99.1%, Avg loss: 0.035051 

11
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.034742 

12
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.034518 

13
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.034288 

14
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.034085 

15
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.033875 

16
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.033681 

17
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.033498 

18
Test Error: 
 Acc

In [26]:
training_time = time.time() - t0

In [27]:
training_time

2755.8562529087067